# Preparation

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random

In [ ]:
sns.set_theme(style='darkgrid')
pd.set_option("display.max_columns", None)

In [ ]:
from pandas.io.excel import read_excel

# Data Collection

In [ ]:
sales = pd.read_excel("D:\Document\AKADEMIK\SMT 7\SKRIPSI\data\data mlg.xlsx", sheet_name='Sales UMC')[['Month', 'Produk X']]
sales

In [ ]:
inflasi = pd.read_excel("D:\Document\AKADEMIK\SMT 7\SKRIPSI\data\data mlg.xlsx", sheet_name='Inflasi')[['Month','Inflasi Transportasi']]
inflasi

In [ ]:
ihk = pd.read_excel("D:\Document\AKADEMIK\SMT 7\SKRIPSI\data\data mlg.xlsx", sheet_name='IHK Transportasi')
ihk

In [ ]:
Harga_Produk_X = pd.read_excel(r"D:\Document\AKADEMIK\SMT 7\SKRIPSI\data\Analisis Produk_X\google trend\Excel Harga Produk X.xlsx")
Harga_Produk_X

In [ ]:
Harga_Produk_X_merek = pd.read_excel(r"D:\Document\AKADEMIK\SMT 7\SKRIPSI\data\Analisis Produk_X\google trend\Excel Harga Produk_X (+merek).xlsx")
Harga_Produk_X_merek

In [ ]:
Produk_X_search = pd.read_excel(r"D:\Document\AKADEMIK\SMT 7\SKRIPSI\data\Analisis Produk_X\google trend\Excel Produk_X (search term).xlsx")
Produk_X_search

In [ ]:
Perusahaan_Z_malang = pd.read_excel(r"D:\Document\AKADEMIK\SMT 7\SKRIPSI\data\Analisis Produk_X\google trend\Excel Perusahaan_Z malang.xlsx")
Perusahaan_Z_malang

In [ ]:
Perusahaan_Z = pd.read_excel(r"D:\Document\AKADEMIK\SMT 7\SKRIPSI\data\Analisis Produk_X\google trend\Excel Perusahaan_Z.xlsx")
Perusahaan_Z

# Data Integration

### Online Search Data + Sales

In [ ]:
df_search = pd.merge(Harga_Produk_X, Harga_Produk_X_merek, on='Month')
df_search = pd.merge(df_search, Produk_X_search, on='Month')
df_search = pd.merge(df_search, Perusahaan_Z_malang, on='Month')
df_search = pd.merge(df_search, Perusahaan_Z, on='Month')
df_search

In [ ]:
df_search.info()

In [ ]:
df_search

In [ ]:
df_search = pd.merge(df_search, sales, on='Month')

In [ ]:
df_search

### Economic Variables + Sales

In [ ]:
df_eco = pd.merge(inflasi, ihk, on='Month')
df_eco

In [ ]:
df_eco.rename(columns={'Inflasi Transportasi':'Inflasi', 
                       'Indeks Harga Konsumen':'IHK'},
              inplace=True)

In [ ]:
df_eco

### Economic Variables + Online Search Data + Sales

In [ ]:
df_full = pd.merge(df_eco, df_search, on='Month')
df_full

# Data Exploratory

#### Missing Value

In [ ]:
df_full.isna().sum()

In [ ]:
df_full[df_full.isna().any(axis=1)]

In [ ]:
df_full.dropna(inplace=True)

In [ ]:
df_full.isna().sum()

In [ ]:
df_full = df_full.reset_index(drop=True)

In [ ]:
df_full.info()

In [ ]:
df_full['Penjualan Produk X'] = df_full['Penjualan Produk X'].astype(int)

#### Descriptive Statistics

In [ ]:
df_full.describe()

In [ ]:
df_cols = df_full.columns
df_cols

#### Data Visualization

##### Time Series

In [ ]:
plt.figure(figsize=(12,4))
plt.xticks(rotation=90)
sns.lineplot(data=df_full, x='Month', y='Penjualan Produk X', marker='o')

for year in range(2014,2026):
    jan_date = pd.Timestamp(f'{year}-01-01')
    plt.axvline(jan_date, color='k', linestyle='--')
    
plt.title('Penjualan Produk X di Dealer U pada Tahun 2014-2025', fontweight='bold')
plt.ylabel('Penjualan Produk X')
plt.tight_layout()
plt.show()

##### Variable Distribution

In [ ]:
df_cols = df_cols[1:10]
df_cols

In [ ]:
fig, axes = plt.subplots(2, 4, figsize=(10,5))
axes=axes.flatten()

for i, col in enumerate(df_cols):
    sns.histplot(df_full, x=df_full[col], ax=axes[i], kde=True, color='#004b66')
    axes[i].set_title(f'Variabel {df_cols[i]}')
    axes[i].set_xlabel('')
    plt.tight_layout()

plt.suptitle("Distribusi Variabel pada Dataset Penjualan Produk X", fontsize=14, fontweight='bold')
plt.tight_layout(rect=[0,0,1,1])
plt.show()

In [ ]:
df_full.head(5)

##### Variable Relationship

In [ ]:
fig, axes = plt.subplots(2, 4, figsize=(10,5))
axes=axes.flatten()

for i, col in enumerate(df_cols):
    sns.scatterplot(df_full,
                    x=df_full['Penjualan Produk X'],
                    y=df_full[col],
                    ax=axes[i],
                    color='#6600cc')
    plt.tight_layout()
    axes[i].set_xlabel('Penjualan Produk X')

plt.suptitle("Hubungan Variabel Independen terhadap Penjualan Produk X", fontsize=14, fontweight='bold')
plt.tight_layout(rect=[0,0,1,1.02])
plt.show()

In [ ]:
plt.figure(figsize=(8,4))
plt.title('Korelasi Antar Variabel (2014-2023)', fontweight='bold')
sns.heatmap(df_full.corr(), annot=True, cmap='viridis', fmt='.2f')
plt.show()

In [ ]:
df_full.corr()

#### Outlier Identification

In [ ]:
fig, axes = plt.subplots(2, 4, figsize=(10,4))
axes=axes.flatten()

for i, col in enumerate(df_cols):
    sns.boxplot(data=df_full,
                x=col,
                ax=axes[i],
                color='#00ff99')
    axes[i].set_title(f'Variabel {df_cols[i]}')
    axes[i].set_xlabel('')
    plt.tight_layout()

plt.suptitle("Boxplot dari Seluruh Variabel pada Dataset Penjualan Produk X", fontsize=14, fontweight='bold')
plt.tight_layout(rect=[0,0,1,1])
plt.show()

In [ ]:
Q1_sales = df_full['Penjualan Produk X'].quantile(0.25)
Q3_sales = df_full['Penjualan Produk X'].quantile(0.75)
IQR_sales = Q3_sales - Q1_sales

outlier_sales = df_full[(df_full['Penjualan Produk X'] > Q3_sales + 1.5*IQR_sales) |
                        (df_full['Penjualan Produk X'] < Q1_sales - 1.5*IQR_sales)]
outlier_sales[['Month', 'Penjualan Produk X']]

In [ ]:
print(f'Q1: {Q1_sales}, Q3: {Q3_sales}, IQR: {IQR_sales}')

## Decomposition

In [ ]:
from statsmodels.tsa.seasonal import STL

In [ ]:
df_full.head()

In [ ]:
target = df_full[['Month', 'Penjualan Produk X']].set_index(keys='Month')
target

In [ ]:
stl = STL(target, period=12)
result = stl.fit()

In [ ]:
seasonal, trend, resid = result.seasonal, result.trend, result.resid

In [ ]:
stl_element = [target, trend, seasonal]

In [ ]:
plt.figure(figsize=(15,12))
ylabel=['Series', 'Trend', 'Seasonality', 'Residual']
for index, element in enumerate(stl_element):
    plt.subplot(4,1,index+1)
    sns.lineplot(element, marker='o')
    plt.ylabel(ylabel[index])
    
    for year in range(2014,2026):
        jan_date = pd.Timestamp(f'{year}-01-01')
        plt.axvline(jan_date, color='k', linestyle='--')
    
plt.subplot(4,1,4)
sns.scatterplot(resid)
plt.ylabel(ylabel[3])
for year in range(2014,2026):
        jan_date = pd.Timestamp(f'{year}-01-01')
        plt.axvline(jan_date, color='k', linestyle='--')

plt.suptitle("Dekomposisi Data Penjualan Produk X", fontsize=16, fontweight='bold')
plt.tight_layout(rect=[0, 0, 1, 1])
plt.show()

### Sales Autocorrelation

In [ ]:
from statsmodels.graphics.tsaplots import plot_pacf, plot_acf
from statsmodels.tsa.stattools import adfuller

#### Uji Stasioneritas

In [ ]:
def adf_test(series):
    result = adfuller(series)
    if result[1] >= 0.05:
        print(f'p-value= {result[1]}\nData Non-Stasioner')
    
    else:
        print(f'p-value= {result[1]}\nData Stasioner')

In [ ]:
df_full

In [ ]:
adf_test(df_full['Penjualan Produk X'])

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(10,4))
ax = ax.flatten()

plot_pacf(df_full['Penjualan Produk X'], lags=12, ax=ax[0])
ax[0].set_title('Plot PACF dari Variabel Penjualan Produk X')

plot_acf(df_full['Penjualan Produk X'], lags=12, ax=ax[1])
ax[1].set_title('Plot ACF dari Variabel Penjualan Produk X')

plt.tight_layout()
plt.show()

##### 

# Model Training (Holt-Winters)

## Prep

In [ ]:
from pmdarima import auto_arima
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error

In [ ]:
hw_train = df_full['Penjualan Produk X'].iloc[:120].copy()
hw_test = df_full['Penjualan Produk X'].tail(14).copy().reset_index(drop=True)

In [ ]:
hw_train

In [ ]:
hw_test

##### 

#### Date Index

In [ ]:
df_eco['Month'].iloc[12:132].reset_index(drop=True)

In [ ]:
train_date = df_eco['Month'].iloc[12:132].reset_index(drop=True)
test_date = df_eco['Month'].loc[132:].reset_index(drop=True)

In [ ]:
train_date

In [ ]:
test_date

##### 

### Holt-Winters

In [ ]:
hw_train[hw_train == 0]

Ada nilai 0, sehingga harus diubah ke nilai positif karena model Holt-Winters yang digunakan adalah model 'multiplicative' yang hanya bisa menerima nilai positif. Maka dari itu, nilai 0 ini akan diubah menjadi 1.

In [ ]:
hw_train[hw_train == 0] = 1

In [ ]:
hw_train[hw_train == 0]

In [ ]:
hw_model_awal = ExponentialSmoothing(hw_train, 
                                     trend='add', 
                                     seasonal='mul', 
                                     seasonal_periods=12).fit()

In [ ]:
print("Smoothing Parameters")
print("Alpha (level):", hw_model_awal.params['smoothing_level'])
print("Beta (trend):", hw_model_awal.params.get('smoothing_trend'))
print("Gamma (seasonal):", hw_model_awal.params.get('smoothing_seasonal'))

print("\nInitial State")
print("Initial level:", hw_model_awal.params['initial_level'])
print("Initial trend:", hw_model_awal.params.get('initial_trend'))
print("Initial seasonal:", hw_model_awal.params.get('initial_seasonal'))

In [ ]:
hw_model_awal.trend

In [ ]:
hw_awal_insample = hw_model_awal.predict(start=0,
                                         end=len(hw_train) - 1)

hw_awal_outsample = hw_model_awal.predict(start=len(hw_train),
                                          end = len(hw_train)
                                                + len (hw_test) 
                                                - 1)

In [ ]:
hw_awal_insample_df = pd.DataFrame({'Y_Actual': hw_train,
                               'Y_Predicted': hw_awal_insample.reset_index(drop=True),
                               'Date': train_date})

hw_awal_outsample_df = pd.DataFrame({'Y_Actual': hw_test,
                                'Y_Predicted': hw_awal_outsample.reset_index(drop=True),
                                'Date': test_date})

In [ ]:
hw_awal_insample_df['Y_Predicted'] = round(hw_awal_insample_df['Y_Predicted']).astype(int)
hw_awal_outsample_df['Y_Predicted'] = round(hw_awal_outsample_df['Y_Predicted']).astype(int)

In [ ]:
hw_awal_insample_df.set_index(keys='Date', inplace=True)
hw_awal_outsample_df.set_index(keys='Date', inplace=True)

In [ ]:
hw_awal_insample_df.head(24)

In [ ]:
# hw Out of Sample
plt.figure(figsize=(10,4))
plt.xticks(rotation=90)
sns.lineplot(data=hw_awal_insample_df, dashes=False, marker='o')
    
plt.title('In Sample Actual vs Predicted Value using Holt-Winters Exponential Smoothing', fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
hw_awal_outsample_df

In [ ]:
# hw Out of Sample
plt.figure(figsize=(10,4))
plt.xticks(rotation=90)
sns.lineplot(data=hw_awal_outsample_df, dashes=False, marker='o')
    
plt.title('Out of Sample Actual vs Predicted Value using Holt-Winters Exponential Smoothing')
plt.tight_layout()
plt.show()

In [ ]:
hw_awal_insample_df

In [ ]:
hw_awal_outsample_df

In [ ]:
df_hw = pd.concat([hw_awal_insample_df, hw_awal_outsample_df])
df_hw

In [ ]:
df_hw.rename(columns={'Y_Actual':'Aktual', 'Y_Predicted':'HW Awal'}, inplace=True)

##### 

In [ ]:
hw_model = ExponentialSmoothing(hw_train, trend='mul', seasonal='mul', seasonal_periods=12).fit()

In [ ]:
print("Smoothing Parameters")
print("Alpha (level):", hw_model.params['smoothing_level'])
print("Beta (trend):", hw_model.params.get('smoothing_trend'))
print("Gamma (seasonal):", hw_model.params.get('smoothing_seasonal'))

print("\nInitial State")
print("Initial level:", hw_model.params['initial_level'])
print("Initial trend:", hw_model.params.get('initial_trend'))
print("Initial seasonal:", hw_model.params.get('initial_seasonal'))

In [ ]:
hw_model.trend

In [ ]:
hw_insample = hw_model.predict(start=0, end=len(hw_train) - 1)
hw_outsample = hw_model.predict(start=len(hw_train), end = len(hw_train) + len (hw_test) - 1)

In [ ]:
hw_insample_df = pd.DataFrame({'Y_Actual': hw_train,
                               'Y_Predicted': hw_insample.reset_index(drop=True),
                               'Date': train_date})

hw_outsample_df = pd.DataFrame({'Y_Actual': hw_test,
                                'Y_Predicted': hw_outsample.reset_index(drop=True),
                                'Date': test_date})

In [ ]:
hw_insample_df['Y_Predicted'] = round(hw_insample_df['Y_Predicted']).astype(int)
hw_outsample_df['Y_Predicted'] = round(hw_outsample_df['Y_Predicted']).astype(int)

In [ ]:
hw_insample_df.set_index(keys='Date', inplace=True)
hw_outsample_df.set_index(keys='Date', inplace=True)

In [ ]:
hw_insample_df.head(24)

In [ ]:
# hw Out of Sample
plt.figure(figsize=(10,4))
plt.xticks(rotation=90)
sns.lineplot(data=hw_insample_df, dashes=False, marker='o')
    
plt.title('In Sample Actual vs Predicted Value using Holt-Winters Exponential Smoothing', fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
hw_outsample_df

In [ ]:
# hw Out of Sample
plt.figure(figsize=(10,4))
plt.xticks(rotation=90)
sns.lineplot(data=hw_outsample_df, dashes=False, marker='o')
    
plt.title('Out of Sample Actual vs Predicted Value using Holt-Winters Exponential Smoothing')
plt.tight_layout()
plt.show()

#### HW Model Evaluation

In [ ]:
# in of Sample
hw_insample_mae = mean_absolute_error(hw_insample_df['Y_Actual'], hw_insample_df['Y_Predicted'])

hw_insample_rmse = mean_squared_error(hw_insample_df['Y_Actual'], hw_insample_df['Y_Predicted'], squared=False)

print(f'Holt-Winters (In Sample) - RMSE: {hw_insample_rmse}, MAE: {hw_insample_mae}')

In [ ]:
# Out of Sample
hw_outsample_mae = mean_absolute_error(hw_outsample_df['Y_Actual'], hw_outsample_df['Y_Predicted'])

hw_outsample_rmse = mean_squared_error(hw_outsample_df['Y_Actual'], hw_outsample_df['Y_Predicted'], squared=False)

print(f'Holt-Winters (Out of Sample) - RMSE: {hw_outsample_rmse}, MAE: {hw_outsample_mae}')

##### 

In [ ]:
df_hw_opt = pd.concat([hw_insample_df, hw_outsample_df])
df_hw_opt = df_hw_opt[['Y_Predicted']]
df_hw_opt

In [ ]:
df_hw = pd.concat([df_hw, df_hw_opt], axis=1)
df_hw

In [ ]:
df_hw.rename(columns={'Y_Predicted':'HW Optimal'}, inplace=True)

##### 

### HW + Expanding Window Forecast

In [ ]:
history = list(hw_train)
predictions = []

for t in range(len(hw_test)):
    model = ExponentialSmoothing(history, trend='mul', seasonal='mul', seasonal_periods=12)
    model_fit = model.fit()

    forecast = model_fit.forecast(steps=1)
    predictions.append(forecast[0])
    history.append(hw_test.iloc[t])

expanding_forecast_df = pd.DataFrame({'Y_Actual' : hw_test.reset_index(drop=True),
                                    'Y_Predicted' : predictions,
                                    'Date' : test_date})

expanding_forecast_df.set_index(keys='Date', inplace=True)
expanding_forecast_df['Y_Predicted'] = round(expanding_forecast_df['Y_Predicted']).astype(int
                                                                                     )
roll_mae = mean_absolute_error(expanding_forecast_df['Y_Actual'], expanding_forecast_df['Y_Predicted'])
roll_rmse = mean_squared_error(expanding_forecast_df['Y_Actual'], expanding_forecast_df['Y_Predicted'], squared=False)

plt.figure(figsize=(10,4))
sns.lineplot(expanding_forecast_df, marker='o', dashes=False)
plt.tight_layout()
plt.show()

print(f"Opt. HW + Expanding Forecast - RMSE = {roll_rmse}, MAE = {roll_mae}")

In [ ]:
expanding_forecast_df

In [ ]:
hw_outs_exp_df = expanding_forecast_df.copy()
hw_outs_exp_df

##### 

In [ ]:
pd.concat([hw_insample_df, hw_outs_exp_df])[['Y_Predicted']]

In [ ]:
df_hw_opt_exp = pd.concat([hw_insample_df, hw_outs_exp_df])
df_hw_opt_exp = df_hw_opt_exp[['Y_Predicted']]
df_hw_opt_exp

In [ ]:
df_hw = pd.concat([df_hw, df_hw_opt_exp], axis=1)
df_hw

In [ ]:
df_hw.rename(columns={'Y_Predicted':'HW Optimal + Expanding Window'}, inplace=True)

In [ ]:
df_hw

In [ ]:
plt.figure(figsize=(10,5))
plt.plot(df_hw['Aktual'], label='Data Aktual', marker='.')
plt.plot(df_hw['HW Optimal + Expanding Window'], label='Data Peramalan', marker='.')

plt.axvline(pd.Timestamp('2023-12-01'), linestyle='--', color='k', alpha=0.5)
plt.title("Peramalan Sales dengan Holt-Winters Exponential Smoothing")
plt.xlabel('Month')
plt.ylabel("Sales")
plt.legend(loc='upper right')
plt.tight_layout()
plt.show()

##### 

#### HW Residual

In [ ]:
hw_insample_df['Residual'] = hw_insample_df['Y_Actual'] - hw_insample_df['Y_Predicted']
hw_insample_df

In [ ]:
plt.figure(figsize=(10,5))
plt.plot(hw_insample_df['Y_Actual'], label='Data Aktual', marker='.')
plt.plot(hw_insample_df['Y_Predicted'], label='Data Peramalan', marker='.')

plt.axvline(pd.Timestamp('2023-12-01'), linestyle='--', color='k', alpha=0.5)
plt.title("Peramalan In-Sample dengan Model Holt-Winters Optimal")
plt.xlabel('Month')
plt.ylabel("Sales")
plt.legend(loc='upper right')
plt.tight_layout()
plt.show()

In [ ]:
hw_outs_exp_df['Residual'] = hw_outs_exp_df['Y_Actual'] - hw_outs_exp_df['Y_Predicted']
hw_outs_exp_df

In [ ]:
plt.figure(figsize=(10,5))
plt.plot(hw_outs_exp_df['Y_Actual'], label='Data Aktual', marker='.')
plt.plot(hw_outs_exp_df['Y_Predicted'], label='Data Peramalan', marker='.')

plt.title("Peramalan Out-of-Sample dengan Model Holt-Winters Optimal + Expanding Window Forecast")
plt.xlabel('Month')
plt.ylabel("Sales")
plt.legend(loc='upper right')
plt.tight_layout()
plt.show()

In [ ]:
pd.concat([hw_insample_df, hw_outs_exp_df])

In [ ]:
hw_ins_res = hw_insample_df['Residual'].reset_index(drop=True)
hw_outs_res = hw_outs_exp_df['Residual'].reset_index(drop=True)

In [ ]:
hw_ins_res

In [ ]:
hw_outs_res

In [ ]:
hw_res = pd.concat([hw_insample_df['Residual'], hw_outs_exp_df['Residual']])

In [ ]:
hw_res

In [ ]:
plt.figure(figsize=(10,4))
sns.lineplot(hw_res, marker='o', label='Residual')

plt.legend(loc='upper right')
plt.tight_layout()
plt.show()

##### 

In [ ]:
df_hw['Residual HW'] = hw_res
df_hw

In [ ]:
df_hw.to_excel('Holt-Winters Dataframe.xlsx')

##### 

# Feature Selection

## Feature Importance

#### Prep

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import export_text
from sklearn.tree import plot_tree
from sklearn.tree import export_graphviz
import graphviz

In [ ]:
lags = [1,2,3]

In [ ]:
df_full['HW Residual'] = hw_res.reset_index(drop=True)

### Random Forest

#### Data Preparation for Feat. Importance

In [ ]:
df_RF = df_full.drop(columns='Month').copy()

In [ ]:
df_RF.columns[0:-1]

In [ ]:
for col in df_RF.columns[0:-1]:
    for lag in lags:
        df_RF[f'{col}_lag{lag}'] = df_RF[col].shift(lag)

In [ ]:
df_RF

In [ ]:
df_RF.isna().sum()

In [ ]:
df_RF['Inflasi_lag1'].fillna(inflasi.iloc[11]['Inflasi Transportasi'], inplace=True)
df_RF['Inflasi_lag2'].fillna(inflasi.iloc[10:12].reset_index()['Inflasi Transportasi'], inplace=True)
df_RF['Inflasi_lag3'].fillna(inflasi.iloc[9:12].reset_index()['Inflasi Transportasi'], inplace=True)

df_RF['IHK_lag1'].fillna(ihk.iloc[11]['Indeks Harga Konsumen'], inplace=True)
df_RF['IHK_lag2'].fillna(ihk.iloc[10:12].reset_index()['Indeks Harga Konsumen'], inplace=True)
df_RF['IHK_lag3'].fillna(ihk.iloc[9:12].reset_index()['Indeks Harga Konsumen'], inplace=True)

df_RF["'Harga Produk X'_lag1"].fillna(Harga_Produk_X.iloc[23]['Harga Produk_X: (East Java)'], inplace=True)
df_RF["'Harga Produk X'_lag2"].fillna(Harga_Produk_X.iloc[22:24].reset_index()['Harga Produk_X: (East Java)'], inplace=True)
df_RF["'Harga Produk X'_lag3"].fillna(Harga_Produk_X.iloc[21:24].reset_index()['Harga Produk_X: (East Java)'], inplace=True)

df_RF["'Harga Produk X (+merek)'_lag1"].fillna(Harga_Produk_X_merek.iloc[23]['Harga Produk_X (+merek): (East Java)'], inplace=True)
df_RF["'Harga Produk X (+merek)'_lag2"].fillna(Harga_Produk_X_merek.iloc[22:24].reset_index()['Harga Produk_X (+merek): (East Java)'], inplace=True)
df_RF["'Harga Produk X (+merek)'_lag3"].fillna(Harga_Produk_X_merek.iloc[21:24].reset_index()['Harga Produk_X (+merek): (East Java)'], inplace=True)

df_RF["'Produk X'_lag1"].fillna(Produk_X_search.iloc[23]['Produk_X search: (East Java)'], inplace=True)
df_RF["'Produk X'_lag2"].fillna(Produk_X_search.iloc[22:24].reset_index()['Produk_X search: (East Java)'], inplace=True)
df_RF["'Produk X'_lag3"].fillna(Produk_X_search.iloc[21:24].reset_index()['Produk_X search: (East Java)'], inplace=True)

df_RF["'Perusahaan Z malang'_lag1"].fillna(malang.iloc[23]['Perusahaan_Z malang: (East Java)'], inplace=True)
df_RF["'Perusahaan Z malang'_lag2"].fillna(malang.iloc[22:24].reset_index()['Perusahaan_Z malang: (East Java)'], inplace=True)
df_RF["'Perusahaan Z malang'_lag3"].fillna(malang.iloc[21:24].reset_index()['Perusahaan_Z malang: (East Java)'], inplace=True)

df_RF["'Perusahaan Z'_lag1"].fillna(Perusahaan_Z.iloc[23]['Perusahaan_Z: (East Java)'], inplace=True)
df_RF["'Perusahaan Z'_lag2"].fillna(Perusahaan_Z.iloc[22:24].reset_index()['Perusahaan_Z: (East Java)'], inplace=True)
df_RF["'Perusahaan Z'_lag3"].fillna(Perusahaan_Z.iloc[21:24].reset_index()['Perusahaan_Z: (East Java)'], inplace=True)

df_RF['Penjualan Produk X_lag1'].fillna(sales.iloc[35]['Produk_X'], inplace=True)
df_RF['Penjualan Produk X_lag2'].fillna(sales.iloc[34:36].reset_index()['Produk_X'], inplace=True)
df_RF['Penjualan Produk X_lag3'].fillna(sales.iloc[33:36].reset_index()['Produk_X'], inplace=True)

In [ ]:
df_RF.isna().sum()

In [ ]:
df_RF.info()

In [ ]:
X_RF = df_RF.drop(columns=['Penjualan Produk X', 'HW Residual'])

In [ ]:
X_RF_train, X_RF_test = X_RF.iloc[:120], X_RF.iloc[120:]
Y_RF_train, Y_RF_test = hw_ins_res, hw_outs_res

In [ ]:
X_RF_train

In [ ]:
X_RF_test

In [ ]:
Y_RF_train

In [ ]:
Y_RF_test

In [ ]:
RF = RandomForestRegressor(n_estimators=100, random_state=42)
RF.fit(X_RF_train, Y_RF_train)

In [ ]:
import os
os.environ["PATH"] += os.pathsep + r"C:\Program Files\Graphviz\bin"

In [ ]:
def tree_structure(model, tree_index=0):
    dot_data = export_graphviz(model.estimators_[tree_index],
                               out_file=None,
                               feature_names=model.feature_names_in_,
                               filled=True,
                               rounded=True,
                               special_characters=True)

    # Render using graphviz
    graph = graphviz.Source(dot_data)
    graph.render(f"Tree {tree_index} Structure", format='png', cleanup=False)  # Saves as tree0.png
    graph.view()  # Opens the image

In [ ]:
RF_importances = RF.feature_importances_
feature = X_RF.columns

In [ ]:
feature_importance_RF = pd.DataFrame({'Feature' : feature,
                                      'RF Importance' : RF_importances})
feature_importance_RF.sort_values(by='RF Importance', ascending=False, inplace=True)
feature_importance_RF

In [ ]:
plt.figure(figsize=(10, 6))
ax = sns.barplot(data=feature_importance_RF,
            x='RF Importance',
            y='Feature',
            palette="Blues_r")

for index, value in enumerate(feature_importance_RF["RF Importance"]):
    ax.text(value, index, f"{value:.3f}", va='center', fontsize=10)

plt.axvline(0.02, linestyle='--', c='r')
plt.axvline(0.05, linestyle='--', c='g')
plt.xlabel("Feature Importance")
plt.ylabel("Features")
plt.title("Feature Importance from Random Forest")
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
ax = sns.barplot(data=feature_importance_RF.head(10),
            x='RF Importance',
            y='Feature',
            palette="Blues_r")

for index, value in enumerate(feature_importance_RF.head(10)["RF Importance"]):
    ax.text(value, index, f"{value:.3f}", va='center', fontsize=10)

plt.axvline(0.02, linestyle='--', c='r')
plt.axvline(0.05, linestyle='--', c='g')
plt.xlabel("Feature Importance")
plt.ylabel("Features")
plt.title("Feature Importance from Random Forest")
plt.show()

#### Feature Importance using SHAP

In [ ]:
import shap

In [ ]:
explainer = shap.TreeExplainer(RF)
shap_values = explainer.shap_values(X_RF_train)

In [ ]:
shap_df = pd.DataFrame(shap_values, columns=X_RF_train.columns)
mean_abs_shap = shap_df.abs().mean().sort_values(ascending=False)
mean_abs_shap.head(10)

In [ ]:
shap_df

In [ ]:
shap_df.to_excel('SHAP Values.xlsx')

In [ ]:
shap_df["'Perusahaan Z'_lag1"].abs().sum()

In [ ]:
mean_abs_shap = mean_abs_shap.reset_index()

In [ ]:
mean_abs_shap.columns = ['Feature', 'MASV']
mean_abs_shap

In [ ]:
plt.figure(figsize=(10, 6))
ax = sns.barplot(data=mean_abs_shap.head(10),
            x='MASV',
            y='Feature',
            palette="Greens_r")

for index, value in enumerate(mean_abs_shap.head(10)["MASV"]):
    ax.text(value, index, f"{value:.3f}", va='center', fontsize=10)

plt.xlabel("MASV")
plt.ylabel("Features")
plt.title("Mean Absolute SHAP Value")
plt.show()

In [ ]:
reduced_feature = ["'Perusahaan Z'_lag1", 'Inflasi_lag1', "'Produk X'_lag1",
                   'Inflasi', "'Harga Produk X'_lag2", 'Inflasi_lag2',
                   "'Harga Produk X (+merek)'_lag1", 'Penjualan Produk X_lag3', "'Harga Produk X'_lag3",
                    "'Perusahaan Z malang'_lag3"]

In [ ]:
X_RF_train_red = X_RF_train[reduced_feature]
X_RF_train_red

In [ ]:
X_RF_test_red = X_RF_test[reduced_feature]
X_RF_test_red

# 

### Feature Importance with Reduced Data

In [ ]:
X_RF_train_red.head()

In [ ]:
RF.fit(X_RF_train_red, Y_RF_train)

In [ ]:
RF_red_importances = RF.feature_importances_
feature_red = X_RF_train_red.columns

In [ ]:
feature_importance_RF_red = pd.DataFrame({'Feature' : feature_red,
                                          'RF Importance (reduced)' : RF_red_importances})
feature_importance_RF_red.sort_values(by='RF Importance (reduced)', ascending=False, inplace=True)
feature_importance_RF_red

In [ ]:
plt.figure(figsize=(10, 6))
ax = sns.barplot(data=feature_importance_RF_red,
            x='RF Importance (reduced)',
            y='Feature',
            palette="Blues_r")

for index, value in enumerate(feature_importance_RF_red["RF Importance (reduced)"]):
    ax.text(value, index, f"{value:.3f}", va='center', fontsize=10)

plt.axvline(0.02, linestyle='--', c='r')
plt.axvline(0.05, linestyle='--', c='g')
plt.xlabel("Feature Importance")
plt.ylabel("Features")
plt.title("Feature Importance with Reduced Data using Random Forest")
plt.show()

##### 

# Data Preprocessing

## Feature Engineering (for RF)

In [ ]:
X_RF_test_red = X_RF_test_red.reset_index(drop=True)
X_RF_test_red

#### 'Month' Feature

In [ ]:
X_RF_train_red['Month'] = df_full['Month'].iloc[:120].reset_index(drop=True).dt.month.copy()

In [ ]:
X_RF_test_red['Month'] = df_full['Month'].tail(14).reset_index(drop=True).dt.month.copy()

In [ ]:
X_RF_train_red.head()

In [ ]:
X_RF_test_red.head()

#### 'Desember' Feature

Hasil dari decomposition menunjukkan adanya 'spike' atau lonjakan penjualan hampir di setiap tahun yang hampir selalu terjadi di bulan Desember, sehingga akan ditambahkan kolom 'Desember' untuk merepresentasikan fenomena tersebut.

In [ ]:
X_RF_train_red['Desember'] = X_RF_train_red['Month'].apply(lambda x: 1 if x == 12 else 0)

In [ ]:
X_RF_train_red.tail()

In [ ]:
X_RF_test_red['Desember'] = X_RF_test_red['Month'].apply(lambda x: 1 if x == 12 else 0)

In [ ]:
X_RF_test_red.tail()

#### 'Year' Feature

Fitur ini ditambahkan karena adanya tren penurunan dari tahun-tahun terdahulu sampai tahun terkini.

In [ ]:
X_RF_train_red['Year'] = df_full['Month'].dt.year.iloc[:120].reset_index(drop=True)

In [ ]:
X_RF_train_red.head()

In [ ]:
X_RF_test_red['Year'] = df_full['Month'].dt.year.tail(14).reset_index(drop=True)

In [ ]:
X_RF_test_red.tail()

In [ ]:
X_RF_train_red.shape

Jumlah fitur terlalu banyak untuk dataset dengan 120 data points, sehingga fitur akan dikurangi.

In [ ]:
RF_FE = RandomForestRegressor(n_estimators=100, random_state=42)
RF_FE.fit(X_RF_train_red, Y_RF_train)

In [ ]:
RF_FE_importances = RF_FE.feature_importances_
feature = X_RF_train_red.columns

In [ ]:
feature_importance_RF_FE = pd.DataFrame({'Feature' : feature,
                                      'RF Importance' : RF_FE_importances})
feature_importance_RF_FE.sort_values(by='RF Importance', ascending=False, inplace=True)
feature_importance_RF_FE

In [ ]:
plt.figure(figsize=(10, 6))
ax = sns.barplot(data=feature_importance_RF_FE,
            x='RF Importance',
            y='Feature',
            palette="Blues_r")

for index, value in enumerate(feature_importance_RF_FE["RF Importance"]):
    ax.text(value, index, f"{value:.3f}", va='center', fontsize=10)

plt.axvline(0.02, linestyle='--', c='r')
plt.axvline(0.05, linestyle='--', c='g')
plt.xlabel("Feature Importance")
plt.ylabel("Features")
plt.title("Feature Importance from Random Forest")
plt.show()

#### Feature Importance using SHAP

In [ ]:
explainer_FE = shap.TreeExplainer(RF_FE)
shap_values_FE = explainer_FE.shap_values(X_RF_train_red)

In [ ]:
shap_FE_df = pd.DataFrame(shap_values_FE, columns=X_RF_train_red.columns)
mean_abs_shap = shap_FE_df.abs().mean().sort_values(ascending=False)
mean_abs_shap

In [ ]:
shap_df.head()

In [ ]:
mean_abs_shap = mean_abs_shap.reset_index()

In [ ]:
mean_abs_shap.columns = ['Feature', 'MASV']
mean_abs_shap

In [ ]:
plt.figure(figsize=(10, 6))
ax = sns.barplot(data=mean_abs_shap,
            x='MASV',
            y='Feature',
            palette="Greens_r")

for index, value in enumerate(mean_abs_shap["MASV"]):
    ax.text(value, index, f"{value:.3f}", va='center', fontsize=10)

plt.xlabel("MASV")
plt.ylabel("Features")
plt.title("Mean Absolute SHAP Value")
plt.show()

In [ ]:
final_feat = ["'Perusahaan Z'_lag1", 'Penjualan Produk X_lag3', 'Inflasi_lag1', 
              'Inflasi', "'Harga Produk X (+merek)'_lag1", "'Harga Produk X'_lag2", 
              'Inflasi_lag2', "'Produk X'_lag1", "'Perusahaan Z malang'_lag3", "'Harga Produk X'_lag3"]

In [ ]:
X_RF_train_red = X_RF_train_red[final_feat]
X_RF_test_red = X_RF_test_red[final_feat]

##### 

# Residual Modeling (Random Forest)

## Prep

In [ ]:
X_RF_train_red.head()

In [ ]:
X_RF_test_red.tail()

In [ ]:
Y_RF_train.head()

In [ ]:
Y_RF_test.tail()

X_RF_train_red = Training predictors for Random Forest  
X_RF_test_red = Testing predictors for Random Forest  
Y_RF_train = Training target for Random Forest  
Y_RF_test = Testing target for Random Forest  

##### 

In [ ]:
RF_res = RandomForestRegressor(n_estimators=100, random_state=42)
RF_res.fit(X_RF_train_red, Y_RF_train)

##### tree_structure(RF_res, 2)

In [ ]:
RF_res_ins = RF_res.predict(X_RF_train_red)
RF_res_outs = RF_res.predict(X_RF_test_red)

In [ ]:
RF_res_ins_df = pd.DataFrame({'Y_Actual': Y_RF_train,
                              'Y_Predicted': RF_res_ins,
                              'Date': train_date})

RF_res_outs_df = pd.DataFrame({'Y_Actual': Y_RF_test,
                               'Y_Predicted': RF_res_outs,
                               'Date': test_date})

In [ ]:
RF_res_ins_df['Y_Predicted'] = round(RF_res_ins_df['Y_Predicted']).astype(int)
RF_res_outs_df['Y_Predicted'] = round(RF_res_outs_df['Y_Predicted']).astype(int)

In [ ]:
RF_res_ins_df.set_index(keys='Date', inplace=True)
RF_res_outs_df.set_index(keys='Date', inplace=True)

In [ ]:
RF_res_ins_df

In [ ]:
plt.figure(figsize=(10,4))
plt.xticks(rotation=90)
sns.lineplot(data=RF_res_ins_df, marker='o')
plt.title('In Sample Actual vs Predicted Residual Value using Random Forest')
plt.show()

In [ ]:
RF_res_outs_df

In [ ]:
plt.figure(figsize=(10,4))
plt.xticks(rotation=90)
sns.lineplot(data=RF_res_outs_df, marker='o')
plt.title('Out of Sample Actual vs Predicted Residual Value using Random Forest')
plt.show()

In [ ]:
RF_res_ins_mae = mean_absolute_error(RF_res_ins_df['Y_Actual'], RF_res_ins_df['Y_Predicted'])
print(f'RF Performance (In Sample) - MAE: {RF_res_ins_mae}')

In [ ]:
RF_res_outs_mae = mean_absolute_error(RF_res_outs_df['Y_Actual'], RF_res_outs_df['Y_Predicted'])
print(f'RF Performance (Out of Sample) - MAE: {RF_res_outs_mae}')

In [ ]:
RF_res_ins_mae = mean_absolute_error(RF_res_ins_df['Y_Actual'], RF_res_ins_df['Y_Predicted'])
RF_res_outs_mae = mean_absolute_error(RF_res_outs_df['Y_Actual'], RF_res_outs_df['Y_Predicted'])

##### 

In [ ]:
df_rf_awal = pd.concat([RF_res_ins_df, RF_res_outs_df])
df_rf_awal.rename(columns={'Y_Actual':'Aktual',
                           'Y_Predicted':'RF Awal'}, inplace=True)
df_rf_awal

##### 

In [ ]:
res_explainer = shap.TreeExplainer(RF_res)
res_shap_values = res_explainer.shap_values(X_RF_train_red)

In [ ]:
res_shap_df = pd.DataFrame(res_shap_values, columns=X_RF_train_red.columns)
res_mean_abs_shap = res_shap_df.abs().mean().sort_values(ascending=False)
res_mean_abs_shap

In [ ]:
res_shap_df.head()

In [ ]:
res_mean_abs_shap = res_mean_abs_shap.reset_index()

In [ ]:
res_mean_abs_shap

In [ ]:
res_mean_abs_shap.columns = ['Feature', 'MASV']
res_mean_abs_shap

In [ ]:
plt.figure(figsize=(10, 6))
ax = sns.barplot(data=res_mean_abs_shap,
            x='MASV',
            y='Feature',
            palette="Greens_r")

for index, value in enumerate(res_mean_abs_shap["MASV"]):
    ax.text(value, index, f"{value:.3f}", va='center', fontsize=10)

plt.xlabel("MASV")
plt.ylabel("Features")
plt.title("Mean Absolute SHAP Value")
plt.show()

##### 

# Hyperparameter Tuning (GridSearch)

In [ ]:
def perform_RF(X_train, X_test, Y_train, Y_test, n_estimators, max_depth, min_samples_split, min_samples_leaf, max_features):
    RF_model = RandomForestRegressor(n_estimators=n_estimators,
                                     max_depth=max_depth,
                                     min_samples_split=min_samples_split,
                                     min_samples_leaf=min_samples_leaf,
                                     max_features=max_features, random_state=42
                                    )
    
    RF_model.fit(X_train, Y_train)
    
    RF_model_insample = RF_model.predict(X_train)
    RF_model_outsample = RF_model.predict(X_test)
    
    RF_model_insample_df = pd.DataFrame({'Y_Actual': Y_train,
                                         'Y_Predicted': RF_model_insample,
                                         'Date': train_date})
    RF_model_outsample_df = pd.DataFrame({'Y_Actual': Y_test,
                                          'Y_Predicted': RF_model_outsample,
                                          'Date': test_date})
        
    RF_model_insample_df['Y_Predicted'] = round(RF_model_insample_df['Y_Predicted']).astype(int)
    RF_model_outsample_df['Y_Predicted'] = round(RF_model_outsample_df['Y_Predicted']).astype(int)
    
    RF_model_insample_df.set_index(keys='Date', inplace=True)
    RF_model_outsample_df.set_index(keys='Date', inplace=True)
    
    plt.figure(figsize=(10,4))
    plt.xticks(rotation=90)
    sns.lineplot(data=RF_model_insample_df, marker='o')
    plt.title('In Sample Actual vs Predicted Value using RF Model')
    plt.show()
    
    plt.figure(figsize=(10,4))
    plt.xticks(rotation=90)
    sns.lineplot(data=RF_model_outsample_df, marker='o')
    plt.title('Out of Sample Actual vs Predicted Value using Random Forest')
    plt.show()
    
    RF_model_insample_mae = mean_absolute_error(RF_model_insample_df['Y_Actual'], RF_model_insample_df['Y_Predicted'])
    RF_model_outsample_mae = mean_absolute_error(RF_model_outsample_df['Y_Actual'], RF_model_outsample_df['Y_Predicted'])
    
    print(f'{n_estimators}, {max_depth}, {min_samples_split}, {min_samples_leaf}, {max_features}, random_state=42')
    print(f'RF Model (In Sample) - MAE: {RF_model_insample_mae}')
    print(f'RF Model (Out of Sample) - MAE: {RF_model_outsample_mae}')

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ParameterGrid

#### Hyp. Optimization with Performance Gap

In [ ]:
param_grid = {'n_estimators': [2, 50, 100, 150, 200, 250], 
              'max_depth': [5, 10, 15, 20], 
              'min_samples_split': [2, 4, 6, 8, 10], 
              'min_samples_leaf': [1, 2, 3, 4, 5], 
              'max_features': [1, 3, 5, 10]}

best_mae = float('inf')
best_params = None
results = []

for params in ParameterGrid(param_grid):
    RF_GS_model = RandomForestRegressor(**params, random_state=42)
    RF_GS_model.fit(X_RF_train_red, Y_RF_train)
    
    RF_GS_insample = RF_GS_model.predict(X_RF_train_red)
    RF_GS_outsample = RF_GS_model.predict(X_RF_test_red)
    
    ins_mae = mean_absolute_error(Y_RF_train, RF_GS_insample)
    outs_mae = mean_absolute_error(Y_RF_test, RF_GS_outsample)
    
    results.append({'params': params,
                    'insample_mae': ins_mae,
                    'outsample_mae': outs_mae})
    
    if outs_mae < best_mae:
        best_mae = outs_mae
        best_params = params
        
print(f"Best parameters: {best_params}")
print(f"Best out-of-sample MAE: {best_mae}")

##### 

In [ ]:
perform_RF(X_RF_train_red, X_RF_test_red, Y_RF_train, Y_RF_test, 50, 5, 8, 1, 5)

In [ ]:
RF_gs = RandomForestRegressor(n_estimators=50, max_depth=5, min_samples_split=8, min_samples_leaf=1, max_features=5, random_state=42)
RF_gs.fit(X_RF_train_red, Y_RF_train)

In [ ]:
RF_gs_ins = RF_gs.predict(X_RF_train_red)
RF_gs_outs = RF_gs.predict(X_RF_test_red)

In [ ]:
RF_gs_ins_df = pd.DataFrame({'Y_Actual': Y_RF_train,
                              'Y_Predicted': RF_gs_ins,
                              'Date': train_date})

RF_gs_outs_df = pd.DataFrame({'Y_Actual': Y_RF_test,
                               'Y_Predicted': RF_gs_outs,
                               'Date': test_date})

In [ ]:
RF_gs_ins_df['Y_Predicted'] = round(RF_gs_ins_df['Y_Predicted']).astype(int)
RF_gs_outs_df['Y_Predicted'] = round(RF_gs_outs_df['Y_Predicted']).astype(int)

In [ ]:
RF_gs_ins_df.set_index(keys='Date', inplace=True)
RF_gs_outs_df.set_index(keys='Date', inplace=True)

In [ ]:
RF_gs_ins_df

In [ ]:
plt.figure(figsize=(10,4))
plt.xticks(rotation=90)
sns.lineplot(data=RF_gs_ins_df, marker='o')
plt.title('In Sample Actual vs Predicted Residual Value using Random Forest')
plt.show()

In [ ]:
RF_gs_outs_df

In [ ]:
df_rf_gs = pd.concat([RF_gs_ins_df, RF_gs_outs_df])[['Y_Predicted']]
df_rf_gs.rename(columns={'Y_Predicted':'RF Grid Search'}, inplace=True)
df_rf_gs

##### 

In [ ]:
perform_RF(X_RF_train_red, X_RF_test_red, Y_RF_train, Y_RF_test, 65, 13, 8, 1, 2)

In [ ]:
RF_opt = RandomForestRegressor(n_estimators=65, max_depth=13, min_samples_split=8, min_samples_leaf=1, max_features=2, random_state=42)
RF_opt.fit(X_RF_train_red, Y_RF_train)

In [ ]:
RF_opt_ins = RF_opt.predict(X_RF_train_red)
RF_opt_outs = RF_opt.predict(X_RF_test_red)

In [ ]:
RF_opt_ins_df = pd.DataFrame({'Y_Actual': Y_RF_train,
                              'Y_Predicted': RF_opt_ins,
                              'Date': train_date})

RF_opt_outs_df = pd.DataFrame({'Y_Actual': Y_RF_test,
                               'Y_Predicted': RF_opt_outs,
                               'Date': test_date})

In [ ]:
RF_opt_ins_df['Y_Predicted'] = round(RF_opt_ins_df['Y_Predicted']).astype(int)
RF_opt_outs_df['Y_Predicted'] = round(RF_opt_outs_df['Y_Predicted']).astype(int)

In [ ]:
RF_opt_ins_df.set_index(keys='Date', inplace=True)
RF_opt_outs_df.set_index(keys='Date', inplace=True)

In [ ]:
RF_opt_ins_df

In [ ]:
plt.figure(figsize=(10,4))
plt.xticks(rotation=90)
sns.lineplot(data=RF_opt_ins_df, marker='o')
plt.title('In Sample Actual vs Predicted Residual Value using Random Forest')
plt.show()

In [ ]:
RF_opt_outs_df

In [ ]:
df_rf_opt = pd.concat([RF_opt_ins_df, RF_opt_outs_df])[['Y_Predicted']]
df_rf_opt.rename(columns={'Y_Predicted':'RF Optimal'}, inplace=True)
df_rf_opt

##### 

In [ ]:
df_forecast_rf = pd.concat([df_rf_awal, df_rf_gs], axis=1)
df_forecast_rf = pd.concat([df_forecast_rf, df_rf_opt], axis=1)
df_forecast_rf

In [ ]:
plt.figure(figsize=(10,5))
plt.plot(df_forecast_rf['Aktual'], label='Data Aktual', marker='o')
plt.plot(df_forecast_rf['RF Optimal'], label='Data Peramalan', marker='o', linestyle='--')

plt.axvline(pd.Timestamp('2023-12-01'), linestyle='--', color='k', alpha=0.5)
plt.title("Peramalan Residual dengan Opt. Random Forest")
plt.xlabel('Month')
plt.ylabel("Residual")
plt.legend(loc='upper right')
plt.tight_layout()
plt.show()

##### Model terbaik:  
RandomForestRegressor(n_estimators=65, max_depth=13, min_samples_split=8, min_samples_leaf=1, max_features=2)  
MAE In-Sample = 2.333, Out-of-Sample = 2.0

##### 

# HW + RF

In [ ]:
hw_insample_df

In [ ]:
hw_outs_exp_df

#### RF Model

In [ ]:
RF_model = RandomForestRegressor(n_estimators=65, max_depth=13, 
                                 min_samples_split=8, min_samples_leaf=1, 
                                 max_features=2, random_state=42)

In [ ]:
RF_model.fit(X_RF_train_red, Y_RF_train)

##### tree_structure(RF_model, 0)

##### tree_structure(RF_model, 1)

##### 

In [ ]:
RF_insample = RF_model.predict(X_RF_train_red)
RF_outsample = RF_model.predict(X_RF_test_red)

In [ ]:
RF_insample_df = pd.DataFrame({'Y_Predicted': RF_insample,
                               'Date': train_date})

RF_outsample_df = pd.DataFrame({'Y_Predicted': RF_outsample,
                                'Date': test_date})

In [ ]:
RF_insample_df['Y_Predicted'] = round(RF_insample_df['Y_Predicted']).astype(int)
RF_outsample_df['Y_Predicted'] = round(RF_outsample_df['Y_Predicted']).astype(int)

In [ ]:
RF_insample_df.set_index(keys='Date', inplace=True)
RF_outsample_df.set_index(keys='Date', inplace=True)

In [ ]:
RF_insample_df

In [ ]:
RF_outsample_df

In [ ]:
hw_insample_df['Y_Predicted']

In [ ]:
hw_rf_ins_df = hw_insample_df[['Y_Actual']]
hw_rf_outs_df = hw_outs_exp_df[['Y_Actual']]

In [ ]:
hw_rf_ins_df['Y_Predicted'] = hw_insample_df['Y_Predicted'] 
                              + RF_insample_df['Y_Predicted']

hw_rf_outs_df['Y_Predicted'] = hw_outs_exp_df['Y_Predicted'] 
                               + RF_outsample_df['Y_Predicted']

In [ ]:
hw_rf_ins_df

In [ ]:
plt.figure(figsize=(10,4))
sns.lineplot(hw_rf_ins_df, marker='o', dashes=False)

plt.title('Actual vs Predicted In Sample using Hybrid HW + RF Model',
          fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
hw_rf_outs_df

In [ ]:
plt.figure(figsize=(10,4))
sns.lineplot(hw_rf_outs_df, marker='o', dashes=False)

plt.title('Actual vs Predicted Out of Sample using Hybrid HW + RF Model', 
          fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
df_hybrid = pd.concat([hw_rf_ins_df, hw_rf_outs_df])
df_hybrid.rename(columns={'Y_Actual':'Aktual',
                          'Y_Predicted':'Model Hibrida'}, inplace=True)
df_hybrid

In [ ]:
plt.figure(figsize=(10,5))
plt.plot(df_hybrid['Aktual'], label='Data Aktual', marker='.')
plt.plot(df_hybrid['Model Hibrida'], label='Data Peramalan', marker='.')

plt.axvline(pd.Timestamp('2023-12-01'), linestyle='--', color='k', alpha=0.5)
plt.title("Peramalan Penjualan dengan Model Hibrida")
plt.xlabel('Month')
plt.ylabel("Penjualan")
plt.legend(loc='upper right')
plt.tight_layout()
plt.show()

# Hybrid Model Evaluation

In [ ]:
# In Sample
hw_rf_insample_mae = mean_absolute_error(hw_rf_ins_df['Y_Actual'], hw_rf_ins_df['Y_Predicted'])

hw_rf_insample_rmse = mean_squared_error(hw_rf_ins_df['Y_Actual'], hw_rf_ins_df['Y_Predicted'], squared=False)

print(f'Hybrid Model 1 (In-Sample) - RMSE: {hw_rf_insample_rmse}, MAE: {hw_rf_insample_mae}')

In [ ]:
# Out of Sample
hw_rf_outsample_mae = mean_absolute_error(hw_rf_outs_df['Y_Actual'], hw_rf_outs_df['Y_Predicted'])

hw_rf_outsample_rmse = mean_squared_error(hw_rf_outs_df['Y_Actual'], hw_rf_outs_df['Y_Predicted'], squared=False)

print(f'Hybrid Model 1 (Out-of-Sample) - RMSE: {hw_rf_outsample_rmse}, MAE: {hw_rf_outsample_mae}')

In [ ]:
df_hybrid['Naive'] = df_hybrid['Aktual'].shift(1)

In [ ]:
df_hybrid

In [ ]:
df_hybrid['Model Hibrida'].iloc[1:120]

In [ ]:
hyb_ins_mae = mean_absolute_error(df_hybrid['Aktual'].iloc[1:120], df_hybrid['Model Hibrida'].iloc[1:120])
hyb_ins_rmse = mean_squared_error(df_hybrid['Aktual'].iloc[1:120], df_hybrid['Model Hibrida'].iloc[1:120], squared=False)

hyb_outs_mae = mean_absolute_error(df_hybrid['Aktual'].iloc[120:], df_hybrid['Model Hibrida'].iloc[120:])
hyb_outs_rmse = mean_squared_error(df_hybrid['Aktual'].iloc[120:], df_hybrid['Model Hibrida'].iloc[120:], squared=False)

In [ ]:
naive_ins_mae = mean_absolute_error(df_hybrid['Aktual'].iloc[1:120], df_hybrid['Naive'].iloc[1:120])
naive_ins_rmse = mean_squared_error(df_hybrid['Aktual'].iloc[1:120], df_hybrid['Naive'].iloc[1:120], squared=False)

naive_outs_mae = mean_absolute_error(df_hybrid['Aktual'].iloc[120:], df_hybrid['Naive'].iloc[120:])
naive_outs_rmse = mean_squared_error(df_hybrid['Aktual'].iloc[120:], df_hybrid['Naive'].iloc[120:], squared=False)

In [ ]:
print(f'Hybrid (In-Sample) - RMSE: {hyb_ins_rmse}, MAE: {hyb_ins_mae}')
print(f'Hybrid (Out-of-Sample) - RMSE: {hyb_outs_rmse}, MAE: {hyb_outs_mae}\n')

print(f'Naive (In-Sample) - RMSE: {naive_ins_rmse}, MAE: {naive_ins_mae}')
print(f'Naive (Out-of-Sample) - RMSE: {naive_outs_rmse}, MAE: {naive_outs_mae}')

##### 

In [ ]:
X_RF_train_red.columns

In [ ]:
X_RF_test_red

#### Best Models:
HW = ExponentialSmoothing(hw_train, trend='mul', seasonal='mul', seasonal_periods=12)

RF = RandomForestRegressor((n_estimators=69, max_depth=17, min_samples_split=8, min_samples_leaf=1, max_features=2)

##### 

# Forecasting

### Data Preparation

In [ ]:
X_2025 = pd.read_excel("D:\Document\AKADEMIK\SMT 7\SKRIPSI\data\data mlg.xlsx", sheet_name='2025 Dataset (rev)')
X_2025.drop(columns='Month', inplace=True)
X_2025

In [ ]:
X_2025.info()

In [ ]:
X_2025.rename(columns={"Perusahaan Z'_lag1" : "'Perusahaan Z'_lag1",
                       "Harga Produk X (+merek)'_lag1" : "'Harga Produk X (+merek)'_lag1",
                       "Harga Produk X'_lag2" : "'Harga Produk X'_lag2",
                       "Produk X'_lag1" : "'Produk X'_lag1",
                       "Perusahaan Z malang'_lag3" : "'Perusahaan Z malang'_lag3",
                       "Harga Produk X'_lag3" : "'Harga Produk X'_lag3"}, inplace=True)
X_2025

##### 

# Forecasting (Mar 2025 - Des 2025)

#### Models:
HW = ExponentialSmoothing(hw_train, trend='mul', seasonal='mul', seasonal_periods=12)

RF = RandomForestRegressor(n_estimators=69, max_depth=17, min_samples_split=8, min_samples_leaf=1, max_features=2)

In [ ]:
forecast_train = pd.concat([hw_train, hw_test], ignore_index=True)
forecast_train

## HW Re-Train

In [ ]:
HW_final = ExponentialSmoothing(forecast_train, trend='mul', seasonal='mul', seasonal_periods=12).fit()

In [ ]:
print("Smoothing Parameters")
print("Alpha (level):", HW_final.params['smoothing_level'])
print("Beta (trend):", HW_final.params.get('smoothing_trend'))
print("Gamma (seasonal):", HW_final.params.get('smoothing_seasonal'))

print("\nInitial State")
print("Initial level:", HW_final.params['initial_level'])
print("Initial trend:", HW_final.params.get('initial_trend'))
print("Initial seasonal:", HW_final.params.get('initial_seasonal'))

In [ ]:
HW_final_insample = HW_final.predict(start=0, end=len(forecast_train)-1)
HW_final_insample

In [ ]:
HW_final_insample_df = pd.DataFrame({'Y_Actual' : forecast_train,
                                     'Y_Predicted' : round(HW_final_insample).astype(int),
                                     'Date' : pd.concat([train_date, test_date], ignore_index=True)
                                    })
HW_final_insample_df.set_index(keys='Date', inplace=True)
HW_final_insample_df['Residual'] = HW_final_insample_df['Y_Actual'] - HW_final_insample_df['Y_Predicted']
HW_final_insample_df

## RF Re-Train

In [ ]:
X_RF = pd.concat([X_RF_train_red, X_RF_test_red], ignore_index=True)
X_RF

In [ ]:
Y_RF = HW_final_insample_df['Residual']
Y_RF

In [ ]:
RF_final = RandomForestRegressor(n_estimators=65, max_depth=13, min_samples_split=8, 
                                 min_samples_leaf=1, max_features=2, random_state=42)

In [ ]:
RF_final.fit(X_RF, Y_RF)

##### tree_structure(RF_final, 0)

In [ ]:
RF_final_ins = RF_final.predict(X_RF)

In [ ]:
RF_final_ins_df = pd.DataFrame({'Y_Actual' : Y_RF.reset_index(drop=True),
                                'Y_Predicted' : RF_final_ins,
                                'Date' : pd.concat([train_date, test_date], ignore_index=True)
                               })
RF_final_ins_df.set_index(keys='Date', inplace=True)
RF_final_ins_df['Y_Predicted'] = round(RF_final_ins_df['Y_Predicted']).astype(int)
RF_final_ins_df

## HW Forecast + RF Residual

In [ ]:
HW_final_insample_df

In [ ]:
RF_final_ins_df

In [ ]:
hw_rf_ins_df = HW_final_insample_df['Y_Predicted'] + RF_final_ins_df['Y_Predicted']
hw_rf_ins_df

## HW Rolling Forecast

In [ ]:
n_forecast = 10
start_date = pd.to_datetime('2025-03-01')
forecast_index = pd.date_range(start=start_date, periods=n_forecast, freq='MS')
forecast_index

In [ ]:
n_forecast = 10
history = list(forecast_train)
predictions = []

for t in range(n_forecast):
    model = ExponentialSmoothing(history, trend='mul', seasonal='mul',
                                 seasonal_periods=12)
    model_fit = model.fit()

    forecast = model_fit.forecast(steps=1)
    predictions.append(forecast[0])
    history.append(forecast[0])

forecast_series = pd.Series(predictions, index=forecast_index)
forecast_series

In [ ]:
round(forecast_series).astype(int)

## Forecast

#### Mar 2025 - May 2025

In [ ]:
X_2025

In [ ]:
X_RF_MarMay2025 = X_2025.iloc[2:5]

In [ ]:
MarMay2025_res = RF_final.predict(X_RF_MarMay2025)
MarMay2025_res = pd.Series(MarMay2025_res, index=forecast_index[0:3])
MarMay2025_res

In [ ]:
final_forecast_MarMay2025 = forecast_series.loc['2025-03-01':'2025-05-01'] + MarMay2025_res
final_forecast_MarMay2025

In [ ]:
# Row 10, 11, 12 in 'forecast' column
X_2025.iloc[[5, 6, 7], X_2025.columns.get_loc('Penjualan Produk X_lag3')] = round(final_forecast_MarMay2025).astype(int).reset_index(drop=True)
X_2025

In [ ]:
final_forecast_MarMay2025

#### Jun 2025 - Aug 2025

In [ ]:
X_RF_JunAug2025 = X_2025.iloc[5:8]

In [ ]:
JunAug2025_res = RF_final.predict(X_RF_JunAug2025)
JunAug2025_res = pd.Series(JunAug2025_res, index=forecast_index[3:6])
JunAug2025_res

In [ ]:
final_forecast_JunAug2025 = forecast_series.loc['2025-06-01':'2025-08-01'] + JunAug2025_res
final_forecast_JunAug2025

In [ ]:
X_2025.iloc[[8, 9, 10], X_2025.columns.get_loc('Penjualan Produk X_lag3')] = round(final_forecast_JunAug2025).astype(int).reset_index(drop=True)
X_2025

In [ ]:
final_forecast_JunAug2025

#### Sep 2025 - Nov 2025

In [ ]:
X_RF_SepNov2025 = X_2025.iloc[8:11]

In [ ]:
SepNov2025_res = RF_final.predict(X_RF_SepNov2025)
SepNov2025_res = pd.Series(SepNov2025_res, index=forecast_index[6:9])
SepNov2025_res

In [ ]:
final_forecast_SepNov2025 = forecast_series.loc['2025-09-01':'2025-11-01'] + SepNov2025_res
final_forecast_SepNov2025

In [ ]:
round(final_forecast_SepNov2025).astype(int)[0]

In [ ]:
X_2025.iloc[11, X_2025.columns.get_loc('Penjualan Produk X_lag3')] = round(final_forecast_SepNov2025).astype(int)[0]
X_2025

In [ ]:
final_forecast_SepNov2025

#### Dec 2025

In [ ]:
X_RF_NovDec2025 = X_2025.iloc[10:12]

In [ ]:
X_RF_NovDec2025

In [ ]:
NovDec2025_res = RF_final.predict(X_RF_NovDec2025)
NovDec2025_res = pd.Series(NovDec2025_res, index=forecast_index[8:10])
NovDec2025_res

In [ ]:
final_forecast_NovDec2025 = forecast_series.loc['2025-11-01':'2025-12-01'] + NovDec2025_res
final_forecast_NovDec2025

In [ ]:
round(final_forecast_NovDec2025).astype(int)

In [ ]:
final_forecast_NovDec2025

In [ ]:
final_forecast = pd.concat([final_forecast_MarMay2025, final_forecast_JunAug2025])
final_forecast = pd.concat([final_forecast, final_forecast_SepNov2025.iloc[0:2]])
final_forecast = pd.concat([final_forecast, final_forecast_NovDec2025])
final_forecast

In [ ]:
final_forecast = round(final_forecast).astype(int)

In [ ]:
final_forecast

In [ ]:
plt.figure(figsize=(10,4))
plt.plot(ertiga.set_index(keys='Month')['Ertiga'], marker='.', label='Data Aktual')
plt.plot(final_forecast, marker='.', label='Data Peramalan')

plt.legend()
plt.title('Penjualan Produk X pada Tahun 2014 - 2025', fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10,4))
plt.plot(pd.concat([ertiga.set_index(keys='Month')['Ertiga'].loc['2024-01-01':], final_forecast]), marker='o', label='Data Aktual')
plt.plot(final_forecast, marker='o', label='Data Peramalan (Hybrid)')

plt.legend()
plt.title('Penjualan Produk X pada Tahun 2024 - 2025', fontweight='bold')
plt.tight_layout()
plt.show()